# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116430e+02     1.492842e+00
 * time: 0.0914149284362793
     1     1.117029e+01     8.384320e-01
 * time: 0.9414229393005371
     2    -1.147040e+01     9.749639e-01
 * time: 1.0898230075836182
     3    -3.384655e+01     7.158844e-01
 * time: 1.2578189373016357
     4    -4.734590e+01     5.599326e-01
 * time: 1.3954100608825684
     5    -5.678913e+01     2.198528e-01
 * time: 1.545083999633789
     6    -5.967231e+01     1.855118e-01
 * time: 1.6596200466156006
     7    -6.088511e+01     7.935174e-02
 * time: 1.766761064529419
     8    -6.141259e+01     4.384559e-02
 * time: 1.8815479278564453
     9    -6.170137e+01     3.221528e-02
 * time: 1.9889049530029297
    10    -6.185135e+01     3.480195e-02
 * time: 2.1043601036071777
    11    -6.198259e+01     2.106756e-02
 * time: 2.2111401557922363
    12    -6.205579e+01     1.585453e-02
 * time: 2.3250491619110107
    13    -6.210047e+01     1.505056e-02
 * time: 2.434066057205

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557745
    AtomicNonlocal      14.8522707
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485381 
    Xc                  -19.3336823

    total               -62.261666453130
